In [912]:
# pdf_path = "test/pdf/dc/form/Ceridian-2-EC-SDS-v2.1.pdf"
# output_pdf_path = "test/target/dc/form/Ceridian-2-EC-SDS-v2.1.pdf"

pdf_path = "test/pdf/dc/form/acuron-label.pdf"
output_pdf_path = "test/target/dc/form/acuron-label.pdf"

# pdf_path = "test/pdf/dc/form/Blue_Book_2023_web.pdf"
# output_pdf_path = "test/target/dc/form/Blue_Book_2023_web.pdf"

# pdf_path = "test/pdf/dc/form/Ceridian-2-EC-Supplemental-Label.pdf"
# output_pdf_path = "test/target/dc/form/Ceridian-2-EC-Supplemental-Label.pdf"

# pdf_path = "test/pdf/dc/form/OMDXE11749.pdf"
# output_pdf_path = "test/target/dc/form/OMDXE11749.pdf"

In [913]:
import fitz
import pandas as pd
import numpy as np
from pprint import pprint
from collections import defaultdict
from typing import Iterable


In [914]:
doc = fitz.open(pdf_path)

In [915]:
doc.page_cropbox(0)

page_width = doc.page_cropbox(0)[2]
page_height = doc.page_cropbox(0)[3]

page_width, page_height

(612.0, 792.0)

In [916]:
pages = []

for page in doc:
    pages.append(page)

## Helper functions & classess

### Functions

In [917]:
def is_within_range(value, target, tolerance):
    return value >= target - tolerance and value <= target + tolerance

def is_same_font_family(font_a, font_b):
    return font_a.split("-")[0] == font_b.split("-")[0]

def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

### TextSegment

In [918]:
class TextSegment(list):

    def __init__(self, spans: list = []):
        super().__init__(spans)
    
    @property
    def text(self):
        return self.__repr__()
    
    @property
    def font(self):
        # Find the most common font
        font_counts = defaultdict(int)
        for span in self:
            font_counts[span.font] += len(span.text)
        return max(font_counts, key=font_counts.get) if font_counts else ""
    
    @property
    def size(self):
        # Find the most common font size
        size_counts = defaultdict(int)
        for span in self:
            size_counts[span.size] += len(span.text)
        return max(size_counts, key=size_counts.get) if size_counts else 0
    
    @property
    def bbox(self):
        # Find the boundary box of all spans
        x0 = min([span.bbox[0] for span in self])
        y0 = min([span.bbox[1] for span in self])
        x1 = max([span.bbox[2] for span in self])
        y1 = max([span.bbox[3] for span in self])
        return (x0, y0, x1, y1)
    
    @property
    def rect(self):
        return fitz.Rect(*self.bbox)
    
    @property 
    def line_height(self):
        # Find the most common line height
        line_height_counts = defaultdict(int)
        for span in self:
            line_height_counts[span.rect.height] += len(span.text)
        return max(line_height_counts, key=line_height_counts.get) if line_height_counts else 0

### TextSpan

In [919]:
class TextSpan():
    _text = ""
    _size = 0
    _flag = 0
    _font = ""
    _color = ""
    _bbox = (0, 0, 0, 0)
    _origin = (0, 0)

    _page_index = 0
    _line_number = 0

    def __init__(self, span, page_index, line_number):
        self._text = span["text"]
        self._size = span["size"]
        self._flag = span["flags"]
        self._font = span["font"]
        self._color = span["color"]
        self._bbox = span["bbox"]
        self._origin = span["origin"]

        self._page_index = page_index
        self._line_number = line_number

    def __repr__(self):
        return self._text.strip()
    
    def __str__(self):
        return self.__repr__()
    
    @property
    def text(self):
        return self._text
    
    @property
    def size(self):
        return self._size
    
    @property
    def flag(self):
        return self._flag
    
    @property
    def font(self):
        return self._font
    
    @property
    def color(self):
        return self._color
    
    @property
    def bbox(self):
        return self._bbox
    
    @property
    def rect(self):
        return fitz.Rect(self._bbox)
    
    @property
    def origin(self):
        return self._origin
    
    @property
    def page_index(self):
        return self._page_index
    
    @property
    def line_number(self):
        return self._line_number
    
    @property
    def is_bold(self):
        return self.flag & 2 ** 4

## Tranform Spans

In [920]:
all_spans = []

for page_index, page in enumerate(doc):
    line_number = 0

    blocks = page.get_text("dict")["blocks"]
    for block_index, block in enumerate(blocks):

        lines = block["lines"]
        for line_index, line in enumerate(lines):

            all_spans.extend([TextSpan(span, page_index, line_number) for span in line["spans"]])
            line_number += 1


## Body text information

In [921]:
all_text_sizes = defaultdict(int)
all_text_fonts = defaultdict(int)
all_text_line_heights = defaultdict(int)

for span in all_spans:
    font = span.font
    text = span.text
    size = span.size
    line_height = span.rect.height

    all_text_sizes[size] += len(text)
    all_text_fonts[font] += len(text)
    all_text_line_heights[line_height] += len(text)


body_text_size = max(all_text_sizes, key=all_text_sizes.get)
body_text_font = max(all_text_fonts, key=all_text_fonts.get)
body_text_line_height = max(all_text_line_heights, key=all_text_line_heights.get)

body_text_size, body_text_font, body_text_line_height

(7.636900424957275, 'FrutigerLTStd-Roman', 9.539276123046875)

In [922]:
all_segments = []

segment = TextSegment([all_spans[0]])

for span_index, span in enumerate(all_spans[1:]):
    last_span = segment[-1]

    # Find the last none empty span in segment
    for s in reversed(segment):
        if s.text.strip():
            last_span = s
            break
    

    is_same_font_fam = is_same_font_family(span.font, last_span.font)
    is_same_font = span.font == last_span.font
    is_same_size = is_within_range(span.size, last_span.size, 0.2)
    is_same_color = span.color == last_span.color
    is_same_flag = span.flag == last_span.flag

    vertical_distance = span.rect.y0 - last_span.rect.y1
    vertical_distance_abs = abs(vertical_distance)
    horizontal_distance = span.rect.x0 - last_span.rect.x1
    horizontal_distance_abs = abs(horizontal_distance)
    left_distance = span.rect.x0 - last_span.rect.x0
    left_distance_abs = abs(left_distance)

    vertical_distance_to_segment = span.rect.y0 - segment.rect.y1
    vertical_distance_to_segment_abs = abs(vertical_distance_to_segment)
    left_distance_to_segment = span.rect.x0 - segment.rect.x0
    left_distance_to_segment_abs = abs(left_distance_to_segment)

    is_on_same_line = span.line_number == last_span.line_number and span.page_index == last_span.page_index
    is_right_next_to = is_on_same_line and horizontal_distance <= 2 # and vertical_distance_abs <= segment.line_height * 0.6
    is_below_next_to = not is_on_same_line and vertical_distance_to_segment <= 1 and vertical_distance_to_segment >= -1 and left_distance_to_segment <= 60


    if_merge = False # the flag
    print(span)

    if is_right_next_to:
        print("right next to")
        if_merge = True

    if is_below_next_to:
        print("below next to")
        if_merge = True

        # if span.is_bold and not last_span.is_bold:
        #     print("bold and not bold")
        #     if_merge = False

        # If height differs too much, don't merge
        if not is_within_range(span.rect.height, last_span.rect.height, segment.line_height * 0.6):
            print("not same height")
            if_merge = False

    
    if if_merge:
        segment.append(span)
    else: 
        all_segments.append(segment)
        segment = TextSegment([span])

all_segments.append(segment)


Sale, use and distribution of this product in Nassau and Suffolk Counties in the State of New York is prohibited.
A Herbicide for Control of Annual Grass and Broadleaf Weeds in Field Corn,
Seed Corn, Silage Corn, Sweet Corn and Yellow Popcorn
below next to
Active Ingredients
:
right next to
S
below next to
-Metolachlor: (CAS No. 87392-12-9) . . . .
right next to
. . . . . .
right next to
. .
right next to
. . . . . . . . . . . . . . . . . . . . . . . .
right next to
. . . . .
right next to
.
right next to
. 23.40%
right next to
Atrazine
below next to
*
right next to
: (CAS No. 1912-24-9)
right next to
. .
right next to
. .
right next to
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
right next to
10.93%
right next to
Mesotrione: (CAS No. 104206-82-8) . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2.60%
below next to
Bicyclopyrone: (CAS No. 352010-68-5)
below next to
. . . . . . . . . . . . . . . . . . . .

In [923]:
for segment_index, segment in enumerate(all_segments):
    color = list(np.random.rand(3,))
    rect = segment.rect

    page.draw_rect(rect, color=color, width=0.6, overlay=True, stroke_opacity=0.5)

    for span_index, span in enumerate(segment):
        page = pages[span.page_index]
        rect = span.rect
        point = rect.tl
        point.x -= 10
        point.y += 8

        page.draw_rect(rect, color=color, width=0.6, overlay=True, stroke_opacity=0.5)
        page.insert_text(point, f"{segment_index}", color=color, fontname="helvetica-bold", fontsize=8, overlay=True)


In [924]:
doc.save(output_pdf_path)